# **Group Assignment** - Bike Sharing

- `instant`: record index
- `dteday` : date
- `season` : season 1:winter, 2:spring, 3:summer, 4:fall
- `yr` : year (0: 2011, 1:2012)
- `mnth` : month ( 1 to 12)
- `hr` : hour (0 to 23)
- `holiday` : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- `weekday` : day of the week
- `workingday` : if day is neither weekend nor holiday is 1, otherwise is 0.
+ `weathersit` : 
	- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
	- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
	- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
	- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- `temp` : Normalized temperature in Celsius. The values are divided to 41 (max)
- `atemp`: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- `hum`: Normalized humidity. The values are divided to 100 (max)
- `windspeed`: Normalized wind speed. The values are divided to 67 (max)
- `casual`: count of casual users
- `registered`: count of registered users
- `cnt`: count of total rental bikes including both casual and registered

In [1]:
import pandas as pd

## PART I: Exploratory Data Analysis

In [4]:
df = pd.read_csv('bike-sharing-hourly.csv')
df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

### Convert the date column to a datetime object

In [6]:
df['dteday'] = pd.to_datetime(df['dteday'])

### Check data is consistent for values where we expect a finite amount of combinations

In [ ]:
print(df['season'].unique())     
print(df['weathersit'].unique())   
print(df['hr'].unique())           

[1 2 3 4]
[1 2 3 4]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


### Create a new time of the day column

In [8]:
def get_time_of_day(hr):
    if 5 <= hr < 12:
        return 'morning'
    elif 12 <= hr < 17:
        return 'afternoon'
    elif 17 <= hr < 21:
        return 'evening'
    else:
        return 'night'
df['time_of_day'] = df['hr'].apply(get_time_of_day)

## PART II: Prediction Model

## PART III: Streamlit dashboard

In [5]:
### This part goes in a separate script ###